In [64]:
import chromadb
from collections import defaultdict

In [ ]:
client = chromadb.PersistentClient(path="../databases/news")
collection_name = "processed_news"
collection = client.get_or_create_collection(name=collection_name)

In [66]:
import numpy as np

all_items = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(all_items['embeddings'])
documents = all_items['documents']
categories = [metadata['category'] for metadata in all_items['metadatas']]

In [67]:
all_items

{'ids': ['0',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '48',
  '49',
  '50',
  '51',
  '52',
  '53',
  '54',
  '55',
  '56',
  '57',
  '58',
  '59',
  '60',
  '61',
  '62',
  '63',
  '64',
  '65',
  '66',
  '67',
  '68',
  '69',
  '70',
  '71',
  '72',
  '73',
  '74',
  '75',
  '76',
  '77',
  '78',
  '79',
  '80',
  '81',
  '82',
  '83',
  '84',
  '85',
  '86',
  '87',
  '88',
  '89',
  '90',
  '91',
  '92',
  '93',
  '94',
  '95',
  '96',
  '97',
  '98',
  '99',
  '100',
  '101',
  '102',
  '103',
  '104',
  '105',
  '106',
  '107',
  '108',
  '109',
  '110',
  '111',
  '112',
  '113',
  '114',
  '115',
  '116',
  '117',
  '118',
  '119',
  '120',
  '121',
  '12

In [68]:
similarity_threshold = 0.2
# frequency = defaultdict(int)

In [69]:
# Find similar vectors above threshold
final_frequency = {}
for item_id, embedding in zip(all_items["ids"], all_items["embeddings"]):
    results = collection.query(
        query_embeddings=[embedding],
        n_results=100
    )
    # Process matches
    ids = [item_id]
    frequency = 0
    for neighbor_id, distance in zip(results["ids"][0], results["distances"][0]):
        if neighbor_id != item_id and distance <= similarity_threshold:
            
            frequency += 1
            ids.append(neighbor_id)
    if frequency > 0:
        if frequency not in final_frequency:
            final_frequency[frequency] = []
        final_frequency[frequency].append(ids)

In [70]:
# final_frequency
# Filter final_frequency to keep only items with frequency > 5
filtered_frequency = {freq: ids_list for freq, ids_list in final_frequency.items() if freq > 5}

# Now filtered_frequency will contain only the keys with frequency > 5
print(filtered_frequency)


{6: [['74', '77', '76', '80', '75', '78', '79'], ['75', '77', '76', '80', '78', '74', '79'], ['76', '77', '80', '75', '78', '74', '79'], ['77', '76', '80', '75', '78', '74', '79'], ['78', '77', '76', '80', '75', '74', '79'], ['79', '77', '76', '80', '75', '78', '74'], ['80', '77', '76', '75', '78', '74', '79'], ['114', '116', '138', '136', '117', '115', '137'], ['115', '136', '117', '137', '116', '114', '138'], ['116', '114', '138', '136', '117', '115', '137'], ['117', '136', '115', '137', '116', '114', '138'], ['119', '121', '123', '120', '124', '125', '122'], ['120', '121', '123', '124', '125', '119', '122'], ['121', '123', '120', '124', '125', '119', '122'], ['122', '121', '123', '120', '124', '125', '119'], ['123', '121', '120', '124', '125', '119', '122'], ['124', '121', '123', '120', '125', '119', '122'], ['125', '121', '123', '120', '124', '119', '122'], ['136', '117', '115', '137', '116', '114', '138'], ['137', '136', '117', '115', '116', '114', '138'], ['138', '116', '114', '1

In [71]:
# Initialize an empty dictionary to store the filtered results
filtered_frequency = {}

# Loop through each frequency and its corresponding list of item IDs
for freq, ids_list in final_frequency.items():
    # We only want to keep frequencies greater than 5
    if freq > 5:
        unique_ids_set = []
        
        # Loop through the list of item ID lists
        for ids in ids_list:
            # Convert the list to a set to eliminate order-based duplicates
            ids_set = set(ids)
            
            # Only add the set to unique_ids_set if it's not already there
            if ids_set not in unique_ids_set:
                unique_ids_set.append(ids_set)
        
        # Store the unique lists (as sets) for this frequency in the filtered dictionary
        filtered_frequency[freq] = unique_ids_set

# Convert the sets back to lists (if needed) and print the result
filtered_frequency = {
    freq: [list(ids_set) for ids_set in ids_list]
    for freq, ids_list in filtered_frequency.items()
}

# Print the filtered results
print(filtered_frequency)


{6: [['74', '75', '77', '78', '79', '76', '80'], ['136', '137', '117', '114', '115', '116', '138'], ['122', '121', '125', '123', '124', '119', '120']], 7: [['108', '109', '105', '110', '106', '104', '107', '111'], ['145', '141', '143', '113', '142', '139', '140', '144'], ['127', '128', '126', '130', '131', '132', '133', '129'], ['158', '157', '155', '162', '161', '159', '160', '156']]}


In [74]:
import random
# Display results
print(f"Vectors with similarity above threshold:")
for count, vector_idss in filtered_frequency.items():
    for vector_ids in vector_idss:
        vector_id = random.choice(vector_ids)
        print(f"Checking ID: {vector_id}")
        
        # Get all items with proper ChromaDB API syntax
        all_items = collection.get(
            include=["metadatas", "documents"]  # ChromaDB automatically returns IDs
        )
        
        # Match IDs with metadata using list indices
        try:
            index = all_items["ids"].index(vector_id)
            metadata = all_items["metadatas"][index]
            document = all_items["documents"][index]
            
            print(f"""
            ID: {vector_id}
            Frequency: {count}
            Documents: {document}
            Metadata: {metadata}
            """)
            
        except ValueError:
            print(f"ID {vector_id} not found in collection")
            continue

Vectors with similarity above threshold:
Checking ID: 80

            ID: 80
            Frequency: 6
            Documents: energy minister chris bowen is lashing out at the former coalition government s  bin fire  gas policy as he clashes with peter dutton on energy 
            Metadata: {'author': 'Mike Foley, Nick Toscano', 'category': 'politics', 'link': 'https://www.theage.com.au/politics/federal/bowen-fired-up-for-election-clash-as-labor-delivers-gas-supply-boost-20250319-p5lknc.html?ref=rss&utm_medium=rss&utm_source=rss_politics_federal', 'published_date': '2025-03-19T13:01:00+00:00', 'sub_category': 'politics', 'title': 'Bowen fired up for election clash as Labor delivers gas supply boost'}
            
Checking ID: 138

            ID: 138
            Frequency: 6
            Documents: the best herald artworks of the week from across the board 
            Metadata: {'category': 'general', 'link': 'https://www.watoday.com.au/politics/federal/cartoons-of-the-week-20250314-p5